In [1]:
!pip install scikit-image
!pip install langchain
!pip install -U boto3
!pip install botocore

  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\chinmaya.sharma\\Anaconda3\\Lib\\site-packages\\~umpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-2bde3a66a51006b2b53eb373ff767a3f.dll'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.32
    Uninstalling botocore-1.24.32:
      Successfully uninstalled botocore-1.24.32
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.5.0
    Uninstalling s3transfer-0.5.0:
      Successfully uninstalled s3transfer-0.5.0
  Attempting uninstall: boto3
    Found existing installation: boto3 1.21.32
    Uninstalling boto3-1.21.32:
      Successfully uninstalled boto3-1.21.32


In [2]:
import boto3
import json
from langchain import PromptTemplate
import os
from skimage import io

Matplotlib is building the font cache; this may take a moment.


In [3]:
#Create you own Credentials for Amazon Bedrock
def getAwsBedrockCreds():
    creds = {}
    creds['aws_access_key_id'] = "key_id"
    creds['aws_secret_access_key'] = "Key"
    creds['region_name'] = "region"
    bedrock_client = boto3.client(
        service_name="bedrock-runtime",
        aws_access_key_id=creds['aws_access_key_id'],
        aws_secret_access_key=creds['aws_secret_access_key'],
        region_name=creds['region_name']
    )  
    return bedrock_client

In [4]:

#Create your own credentials for Amazon Rekognition
def getAwsRekognitionCreds():
    creds = {}
    creds['aws_access_key_id'] = "key_id"
    creds['aws_secret_access_key'] = "key"
    creds['region_name'] = "region"
    rek_client = boto3.client('rekognition', 
                              aws_access_key_id=creds['aws_access_key_id'],
                              aws_secret_access_key=creds['aws_secret_access_key'],
                              region_name=creds['region_name']
                             )
    return rek_client

In [5]:
def interactWithLLM(prompt,llm_type):
    bedrock_client = getAwsBedrockCreds()
    if llm_type == 'titan':
        print("**THE LLM TYPE IS -->" + llm_type)
        #Test for invoke model begins
        parameters = {
            "maxTokenCount":512,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        }
        body = json.dumps({"inputText": prompt, "textGenerationConfig": parameters})
        modelId = "amazon.titan-tg1-large" # change this to use a different version from the model provider
        accept = "application/json"
        contentType = "application/json"
        
        response = bedrock_client.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        
        response_body = json.loads(response.get("body").read())
        response_text_titan = response_body.get("results")[0].get("outputText")
        
        return response_text_titan
    
    elif llm_type == 'claude':
        print("**THE LLM TYPE IS -->" + llm_type)
        body = json.dumps({"prompt": prompt,
                           "max_tokens_to_sample":300,
                           "temperature":1,
                           "top_k":250,
                           "top_p":0.999,
                           "stop_sequences":[]
                          }) 
        modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
        accept = 'application/json'
        contentType = 'application/json'
        
        response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read())
        
        response_text_claud = response_body.get('completion')
        
        return response_text_claud

In [6]:
def imageAnalyzer(input_img):
    
    rek_client = getAwsRekognitionCreds()
    print(input_img)
    
    label_names = ''
    
    with open(input_img, 'rb') as image:
        response = rek_client.detect_text(Image={'Bytes': image.read()})
    textDetections = response['TextDetections']
    print(f'Found {len(textDetections)} texts in the image:')
    for text in textDetections:
        name = text['DetectedText']
        confidence = text['Confidence']
        #print(f'> Text "{name}" with confidence {confidence:.2f}')
        if confidence>95:
            print(name + "|" + str(confidence))
            label_names = label_names + name + ","

    with open(input_img, 'rb') as image:
        response = rek_client.detect_labels(Image={'Bytes': image.read()})

    labels = response['Labels']
    print(f'Found {len(labels)} labels in the image:')
    for label in labels:
        name = label['Name']
        confidence = label['Confidence']
        #print(f'> Label "{name}" with confidence {confidence:.2f}')
        if confidence>95:
            print(name + "|" + str(confidence))
            label_names = label_names + name + ","


    return label_names

In [7]:
input_img = 'MicrosoftTeams-image.png'
img = io.imread(input_img)

In [8]:
label_names = imageAnalyzer(input_img)

MicrosoftTeams-image.png
Found 56 texts in the image:
Database Instances|99.8788833618164
User|99.93124389648438
process|99.90373992919922
SGA|99.77677154541016
Database|99.07947540283203
Redo|100.0
Buffer|99.98965454101562
Log|99.89175415039062
Buffer|99.41127014160156
Cache|100.0
Shared|99.75936889648438
Pool|100.0
Server|99.90956115722656
Java|99.8982162475586
Large|99.89783477783203
Process|99.12545776367188
Pool|100.0
Pool|100.0
Background Processes|99.96189880371094
Database|99.43597412109375
Redo|99.5974349975586
Data|99.66622924804688
Control|99.300537109375
Log|99.65194702148438
Files|99.41419982910156
Files|99.71934509277344
Files|100.0
Database|99.93102264404297
Instances|99.82674407958984
User|99.93124389648438
process|99.90373992919922
SGA|99.77677154541016
Database|99.07947540283203
Redo|100.0
Buffer|99.98965454101562
Log|99.89175415039062
Buffer|99.41127014160156
Cache|100.0
Shared|99.75936889648438
Pool|100.0
Server|99.90956115722656
Java|99.8982162475586
Large|99.89783

In [9]:
prompt_claude = """
Human:  Here are the comma seperated list of labels/objects seen in the image:
{labels}

Please Explain the input image 
Assistant:
"""

In [10]:
prompt_template_for_summary_generate = PromptTemplate.from_template(prompt_claude)
prompt_data_for_summary_generate = prompt_template_for_summary_generate.format(labels=label_names)
llm_type = 'titan'
print("prompt_data_for_summary_generate : ->" + prompt_data_for_summary_generate)

response_text = interactWithLLM(prompt_data_for_summary_generate,llm_type)
print('response_text --- \n' + response_text)

prompt_data_for_summary_generate : ->
Human:  Here are the comma seperated list of labels/objects seen in the image:
Database Instances,User,process,SGA,Database,Redo,Buffer,Log,Buffer,Cache,Shared,Pool,Server,Java,Large,Process,Pool,Pool,Background Processes,Database,Redo,Data,Control,Log,Files,Files,Files,Database,Instances,User,process,SGA,Database,Redo,Buffer,Log,Buffer,Cache,Shared,Pool,Server,Java,Large,Process,Pool,Pool,Background,Processes,Database,Redo,Data,Control,Log,Files,Files,Files,Diagram,UML Diagram,

Please Explain the input image 
Assistant:

**THE LLM TYPE IS -->titan
response_text --- 

The input image you provided is a diagram depicting various components and processes related to a database system. Here is a list of the labels and objects depicted in the image:

1. Database Instances: These represent individual database instances running on the server.
2. User: These represent users accessing the database system.
3. Process: These represent the processes associated

In [14]:
prompt_titan = """
Human:  Here are the comma seperated list of labels/objects seen in the image:
{labels}

Please Explain the input image and describe in details how the flow goes from one label to another
Assistant:
"""

In [15]:
prompt_template_for_summary_generate = PromptTemplate.from_template(prompt_titan)
prompt_data_for_summary_generate = prompt_template_for_summary_generate.format(labels=label_names)
llm_type = 'titan'
print("prompt_data_for_summary_generate : ->" + prompt_data_for_summary_generate)

response_text = interactWithLLM(prompt_data_for_summary_generate,llm_type)
print('response_text --- \n' + response_text)

prompt_data_for_summary_generate : ->
Human:  Here are the comma seperated list of labels/objects seen in the image:
Database Instances,User,process,SGA,Database,Redo,Buffer,Log,Buffer,Cache,Shared,Pool,Server,Java,Large,Process,Pool,Pool,Background Processes,Database,Redo,Data,Control,Log,Files,Files,Files,Database,Instances,User,process,SGA,Database,Redo,Buffer,Log,Buffer,Cache,Shared,Pool,Server,Java,Large,Process,Pool,Pool,Background,Processes,Database,Redo,Data,Control,Log,Files,Files,Files,Diagram,UML Diagram,

Please Explain the input image and describe in details how the flow goes from one label to another
Assistant:

**THE LLM TYPE IS -->titan
response_text --- 

The input image is a flowchart that depicts the flow of data and processes between different components of a database system. Here is a detailed explanation of how the flow goes from one label to another:

1. Database Instances: This label represents the database instances that are part of the system. It could include